# Economics Countries Master Dataset

**Purpose**: Create a consolidated country-year economic structure dataset for merging with conflict data.

**Output**: `economics-countries-master.csv`

**Coverage**: 
- 220 countries
- 1970-2023 (54 years)
- Sector percentages: Primary, Secondary, Tertiary
- Tourism % (2008-2023)
- GDP in USD

**Sources**:
1. World Bank GDP sectoral breakdown (sectoral composition)
2. UN Tourism SDG 8.9.1 (tourism %)
3. World Bank Development Indicators (GDP in USD)

In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print("Libraries loaded successfully")

Libraries loaded successfully


## Step 1: Load World Bank GDP Sectoral Data

Loading the GDP breakdown by sector. Header starts at row 3 (row index 2).

In [4]:
# Load World Bank GDP sectoral data
df_gdp_sectoral = pd.read_excel(
    '../raw-data/World_Bank/Download-GDPcurrent-NCU-countries.xlsx',
    sheet_name=0,
    header=2
)

print(f"Loaded {len(df_gdp_sectoral):,} rows")
print(f"\nColumns: {list(df_gdp_sectoral.columns[:10])}...")
print(f"\nIndicators:")
for ind in df_gdp_sectoral['IndicatorName'].unique():
    count = len(df_gdp_sectoral[df_gdp_sectoral['IndicatorName'] == ind])
    print(f"  - {ind}: {count} countries")

df_gdp_sectoral.head()

Loaded 3,714 rows

Columns: ['CountryID', 'Country', 'Currency', 'IndicatorName', 1970, 1971, 1972, 1973, 1974, 1975]...

Indicators:
  - Final consumption expenditure: 220 countries
  - Household consumption expenditure (including Non-profit institutions serving households): 219 countries
  - General government final consumption expenditure: 219 countries
  - Gross capital formation: 219 countries
  - Gross fixed capital formation (including Acquisitions less disposals of valuables): 219 countries
  - Exports of goods and services: 219 countries
  - Imports of goods and services: 218 countries
  - Gross Domestic Product (GDP): 220 countries
  - Agriculture, hunting, forestry, fishing (ISIC A-B): 217 countries
  - Mining, Manufacturing, Utilities (ISIC C-E): 220 countries
  - Manufacturing (ISIC D): 220 countries
  - Construction (ISIC F): 220 countries
  - Wholesale, retail trade, restaurants and hotels (ISIC G-H): 220 countries
  - Transport, storage and communication (ISIC I): 220 c

,CountryID,Country,Currency,IndicatorName,1970,1971,1972,1973,1974,1975,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,4,Afghanistan,Afghani,Final consumption expenditure,76097244.0,82197023.0,69497573.0,73697424.0,89296879.0,97796585.0,...,1.154152e+12,1.165048e+12,1.249789e+12,1.265019e+12,1.501280e+12,1.662108e+12,1.756194e+12,1.373045e+12,1.533160e+12,1.609344e+12
1,4,Afghanistan,Afghani,Household consumption expenditure (including N...,70967106.0,76655665.0,64812356.0,68729071.0,83276881.0,91203574.0,...,9.053795e+11,9.152604e+11,9.702025e+11,9.937284e+11,1.217000e+12,1.338629e+12,1.416826e+12,1.107016e+12,1.252786e+12,1.321735e+12
2,4,Afghanistan,Afghani,General government final consumption expenditure,5130138.0,5541358.0,4685217.0,4968352.0,6019998.0,6593011.0,...,2.487723e+11,2.497878e+11,2.795870e+11,2.712908e+11,2.842798e+11,3.234784e+11,3.393690e+11,2.660288e+11,2.803736e+11,2.876090e+11
3,4,Afghanistan,Afghani,Gross capital formation,4299850.0,4499842.0,4699836.0,5699800.0,8499703.0,10399636.0,...,1.600561e+11,1.657882e+11,1.582441e+11,1.796258e+11,1.938693e+11,1.892456e+11,1.758400e+11,1.624859e+11,2.139298e+11,2.056450e+11
4,4,Afghanistan,Afghani,Gross fixed capital formation (including Acqui...,4299850.0,4499842.0,4699836.0,5699800.0,8499703.0,10399636.0,...,1.600561e+11,1.657882e+11,1.582441e+11,1.796258e+11,1.938693e+11,1.892456e+11,1.758400e+11,1.624859e+11,2.139298e+11,2.056450e+11


## Step 2: Reshape from Wide to Long Format

Convert year columns (1970-2023) into rows for easier processing.

In [5]:
# Get year columns (all numeric columns from 1970 onwards)
year_columns = [col for col in df_gdp_sectoral.columns if isinstance(col, (int, float)) and col >= 1970]

print(f"Year range: {min(year_columns)} to {max(year_columns)}")
print(f"Total years: {len(year_columns)}")

# Reshape to long format
df_long = df_gdp_sectoral.melt(
    id_vars=['CountryID', 'Country', 'Currency', 'IndicatorName'],
    value_vars=year_columns,
    var_name='Year',
    value_name='Value'
)

# Convert Year to integer
df_long['Year'] = df_long['Year'].astype(int)

print(f"\nReshaped to {len(df_long):,} rows")
df_long.head(10)

Year range: 1970 to 2023
Total years: 54

Reshaped to 200,556 rows


,CountryID,Country,Currency,IndicatorName,Year,Value
0,4,Afghanistan,Afghani,Final consumption expenditure,1970,76097244.0
1,4,Afghanistan,Afghani,Household consumption expenditure (including N...,1970,70967106.0
2,4,Afghanistan,Afghani,General government final consumption expenditure,1970,5130138.0
3,4,Afghanistan,Afghani,Gross capital formation,1970,4299850.0
4,4,Afghanistan,Afghani,Gross fixed capital formation (including Acqui...,1970,4299850.0
5,4,Afghanistan,Afghani,Exports of goods and services,1970,7699731.0
6,4,Afghanistan,Afghani,Imports of goods and services,1970,9399672.0
7,4,Afghanistan,Afghani,Gross Domestic Product (GDP),1970,78697146.0
8,4,Afghanistan,Afghani,"Agriculture, hunting, forestry, fishing (ISIC ...",1970,39539568.0
9,4,Afghanistan,Afghani,"Mining, Manufacturing, Utilities (ISIC C-E)",1970,17121342.0


## Step 3: Pivot Indicators to Columns

Each indicator becomes a column for easier calculation.

In [6]:
# Pivot so each indicator is a column
df_pivot = df_long.pivot_table(
    index=['CountryID', 'Country', 'Currency', 'Year'],
    columns='IndicatorName',
    values='Value',
    aggfunc='first'
).reset_index()

print(f"Pivoted to {len(df_pivot):,} rows (country-years)")
print(f"Columns: {len(df_pivot.columns)}")
print(f"\nCountries: {df_pivot['Country'].nunique()}")
print(f"Year range: {df_pivot['Year'].min()} to {df_pivot['Year'].max()}")

df_pivot.head()

Pivoted to 10,936 rows (country-years)
Columns: 21

Countries: 220
Year range: 1970 to 2023


IndicatorName,CountryID,Country,Currency,Year,"Agriculture, hunting, forestry, fishing (ISIC A-B)",Changes in inventories,Construction (ISIC F),Exports of goods and services,Final consumption expenditure,General government final consumption expenditure,...,Gross capital formation,Gross fixed capital formation (including Acquisitions less disposals of valuables),Household consumption expenditure (including Non-profit institutions serving households),Imports of goods and services,Manufacturing (ISIC D),"Mining, Manufacturing, Utilities (ISIC C-E)",Other Activities (ISIC J-P),Total Value Added,"Transport, storage and communication (ISIC I)","Wholesale, retail trade, restaurants and hotels (ISIC G-H)"
0,4,Afghanistan,Afghani,1970,39539568.0,NaN,2126877.0,7699731.0,76097244.0,5130138.0,...,4299850.0,4299850.0,70967106.0,9399672.0,16823905.0,17121342.0,5806393.0,78697995.0,3814182.0,10289633.0
1,4,Afghanistan,Afghani,1971,41399045.0,NaN,2226881.0,8999685.0,82197023.0,5541358.0,...,4499842.0,4499842.0,76655665.0,13299535.0,17614891.0,17926313.0,6079390.0,82397913.0,3993406.0,10772878.0
2,4,Afghanistan,Afghani,1972,36072594.0,NaN,1940340.0,10599629.0,69497573.0,4685217.0,...,4699836.0,4699836.0,64812356.0,12999545.0,15348652.0,15620008.0,5297214.0,71798259.0,3479600.0,9388503.0
3,4,Afghanistan,Afghani,1973,39187835.0,NaN,2108025.0,10099647.0,73697424.0,4968352.0,...,5699800.0,5699800.0,68729071.0,11499598.0,16674526.0,16969323.0,5754861.0,77998114.0,3780546.0,10197524.0
4,4,Afghanistan,Afghani,1974,48736341.0,NaN,2621489.0,13599525.0,89296879.0,6019998.0,...,8499703.0,8499703.0,83276881.0,14399497.0,20736085.0,21102687.0,7156623.0,96997652.0,4700644.0,12679868.0


## Step 4: Calculate Sector Percentages

**Sector Definitions**:
- **Primary**: Agriculture + Mining (extractive/agricultural)
- **Secondary**: Manufacturing + Construction (industrial)
- **Tertiary**: Trade/Retail + Transport + Other Services (services)

All calculated as % of Total Value Added.

In [7]:
# Shorter column names for easier reference
df_pivot.columns = df_pivot.columns.str.strip()

# Map long indicator names to short names
col_map = {
    'Agriculture, hunting, forestry, fishing (ISIC A-B)': 'Agriculture',
    'Mining, Manufacturing, Utilities (ISIC C-E)': 'Mining_Manuf_Util',
    'Manufacturing (ISIC D)': 'Manufacturing',
    'Construction (ISIC F)': 'Construction',
    'Wholesale, retail trade, restaurants and hotels (ISIC G-H)': 'Trade_Retail',
    'Transport, storage and communication (ISIC I)': 'Transport',
    'Other Activities (ISIC J-P)': 'Other_Activities',
    'Total Value Added': 'Total_Value_Added',
    'Gross Domestic Product (GDP)': 'GDP'
}

df_pivot = df_pivot.rename(columns=col_map)

# Calculate Mining (Mining+Manuf+Util - Manufacturing)
df_pivot['Mining'] = df_pivot['Mining_Manuf_Util'] - df_pivot['Manufacturing']

# Calculate sector components
df_pivot['Primary_Value'] = df_pivot['Agriculture'] + df_pivot['Mining']
df_pivot['Secondary_Value'] = df_pivot['Manufacturing'] + df_pivot['Construction']
df_pivot['Tertiary_Value'] = df_pivot['Trade_Retail'] + df_pivot['Transport'] + df_pivot['Other_Activities']

# Calculate percentages (relative to Total Value Added)
df_pivot['Primary_%'] = (df_pivot['Primary_Value'] / df_pivot['Total_Value_Added']) * 100
df_pivot['Secondary_%'] = (df_pivot['Secondary_Value'] / df_pivot['Total_Value_Added']) * 100
df_pivot['Tertiary_%'] = (df_pivot['Tertiary_Value'] / df_pivot['Total_Value_Added']) * 100

# Validation: check that percentages sum to ~100%
df_pivot['Total_%'] = df_pivot['Primary_%'] + df_pivot['Secondary_%'] + df_pivot['Tertiary_%']

print("Sample calculations:")
print(df_pivot[['Country', 'Year', 'Primary_%', 'Secondary_%', 'Tertiary_%', 'Total_%']].head(10))

print("\nValidation - Total % distribution:")
print(df_pivot['Total_%'].describe())

Sample calculations:
IndicatorName      Country  Year  Primary_%  Secondary_%  Tertiary_%  Total_%
0              Afghanistan  1970  50.620102    24.080387   25.299511    100.0
1              Afghanistan  1971  50.620781    24.080430   25.298789    100.0
2              Afghanistan  1972  50.619542    24.079960   25.300498    100.0
3              Afghanistan  1973  50.619983    24.080776   25.299241    100.0
4              Afghanistan  1974  50.622816    24.080556   25.296628    100.0
5              Afghanistan  1975  50.615827    24.078547   25.305626    100.0
6              Afghanistan  1976  50.621305    24.083222   25.295472    100.0
7              Afghanistan  1977  50.631319    24.079900   25.288781    100.0
8              Afghanistan  1978  50.594856    24.072517   25.332627    100.0
9              Afghanistan  1979  50.637742    24.097251   25.265008    100.0

Validation - Total % distribution:
count    10717.000000
mean       100.025045
std          0.673180
min         79.3897

## Step 5: Load and Merge UN Tourism Data

Tourism as % of GDP (2008-2023 only, 125 countries).

In [8]:
# Load UN Tourism data
df_tourism = pd.read_excel(
    '../raw-data/UN_Tourism/UN_Tourism_8_9_1_TDGDP_04_2025.xlsx',
    sheet_name='SDG 8.9.1',
    header=0
)

print(f"Loaded {len(df_tourism):,} tourism records")
print(f"Countries: {df_tourism['GeoAreaName'].nunique()}")
print(f"Year range: {df_tourism['TimePeriod'].min()} to {df_tourism['TimePeriod'].max()}")

# Select relevant columns and rename
df_tourism_clean = df_tourism[['GeoAreaName', 'TimePeriod', 'Value']].copy()
df_tourism_clean.columns = ['Country', 'Year', 'Tourism_%']

print("\nSample tourism data:")
print(df_tourism_clean.head(10))

Loaded 1,243 tourism records
Countries: 125
Year range: 2008 to 2023

Sample tourism data:
   Country  Year  Tourism_%
0  Albania  2008    2.75707
1  Albania  2009    2.66869
2  Albania  2010    2.81234
3  Albania  2011    2.53477
4  Albania  2012    2.35697
5  Albania  2013    2.32137
6  Albania  2014    2.32641
7  Albania  2015    2.44379
8  Albania  2016    2.63697
9  Albania  2017    2.83579


In [9]:
# Merge tourism data with main dataset
df_master = df_pivot.merge(
    df_tourism_clean,
    on=['Country', 'Year'],
    how='left'
)

print(f"After merging tourism: {len(df_master):,} rows")
print(f"\nTourism data coverage:")
print(f"  - Records with tourism data: {df_master['Tourism_%'].notna().sum():,}")
print(f"  - Records without tourism data: {df_master['Tourism_%'].isna().sum():,}")

# Show countries with tourism data
countries_with_tourism = df_master[df_master['Tourism_%'].notna()]['Country'].nunique()
print(f"  - Countries with tourism data: {countries_with_tourism}")

After merging tourism: 10,936 rows

Tourism data coverage:
  - Records with tourism data: 1,128
  - Records without tourism data: 9,808
  - Countries with tourism data: 115


## Step 6: Add GDP in USD

From World Bank Development Indicators for cross-country comparisons.

In [10]:
# Load World Bank Development Indicators
df_dev_ind = pd.read_csv('../raw-data/World_Bank/world_bank_development_indicators.csv')

print(f"Loaded {len(df_dev_ind):,} development indicator records")
print(f"Countries: {df_dev_ind['country'].nunique()}")

# Extract year from date column
df_dev_ind['Year'] = pd.to_datetime(df_dev_ind['date']).dt.year

# Select GDP in USD column
df_gdp_usd = df_dev_ind[['country', 'Year', 'GDP_current_US']].copy()
df_gdp_usd.columns = ['Country', 'Year', 'GDP_USD']

print("\nSample GDP USD data:")
print(df_gdp_usd.head(10))

Loaded 17,272 development indicator records
Countries: 274

Sample GDP USD data:
       Country  Year       GDP_USD
0  Afghanistan  1960  5.377778e+08
1  Afghanistan  1961  5.488889e+08
2  Afghanistan  1962  5.466667e+08
3  Afghanistan  1963  7.511112e+08
4  Afghanistan  1964  8.000000e+08
5  Afghanistan  1965  1.006667e+09
6  Afghanistan  1966  1.400000e+09
7  Afghanistan  1967  1.673333e+09
8  Afghanistan  1968  1.373333e+09
9  Afghanistan  1969  1.408889e+09


In [11]:
# Merge GDP USD with master dataset
df_master = df_master.merge(
    df_gdp_usd,
    on=['Country', 'Year'],
    how='left'
)

print(f"After merging GDP USD: {len(df_master):,} rows")
print(f"\nGDP USD coverage:")
print(f"  - Records with GDP USD: {df_master['GDP_USD'].notna().sum():,}")
print(f"  - Records without GDP USD: {df_master['GDP_USD'].isna().sum():,}")

After merging GDP USD: 10,936 rows

GDP USD coverage:
  - Records with GDP USD: 8,163
  - Records without GDP USD: 2,773


## Step 7: Create Final Master Dataset

Select and order columns for the final output.

In [12]:
# Select final columns
df_final = df_master[[
    'Country',
    'Year',
    'Primary_%',
    'Secondary_%',
    'Tertiary_%',
    'Tourism_%',
    'GDP_USD'
]].copy()

# Round percentages to 2 decimal places
df_final['Primary_%'] = df_final['Primary_%'].round(2)
df_final['Secondary_%'] = df_final['Secondary_%'].round(2)
df_final['Tertiary_%'] = df_final['Tertiary_%'].round(2)
df_final['Tourism_%'] = df_final['Tourism_%'].round(2)

# Sort by Country and Year
df_final = df_final.sort_values(['Country', 'Year']).reset_index(drop=True)

print(f"Final dataset: {len(df_final):,} rows × {len(df_final.columns)} columns")
print(f"\nCountries: {df_final['Country'].nunique()}")
print(f"Year range: {df_final['Year'].min()} - {df_final['Year'].max()}")

df_final.head(20)

Final dataset: 10,936 rows × 7 columns

Countries: 220
Year range: 1970 - 2023


,Country,Year,Primary_%,Secondary_%,Tertiary_%,Tourism_%,GDP_USD
0,Afghanistan,1970,50.62,24.08,25.30,NaN,1.748887e+09
1,Afghanistan,1971,50.62,24.08,25.30,NaN,1.831109e+09
2,Afghanistan,1972,50.62,24.08,25.30,NaN,1.595555e+09
3,Afghanistan,1973,50.62,24.08,25.30,NaN,1.733333e+09
4,Afghanistan,1974,50.62,24.08,25.30,NaN,2.155555e+09
5,Afghanistan,1975,50.62,24.08,25.31,NaN,2.366667e+09
6,Afghanistan,1976,50.62,24.08,25.30,NaN,2.555556e+09
7,Afghanistan,1977,50.63,24.08,25.29,NaN,2.953333e+09
8,Afghanistan,1978,50.59,24.07,25.33,NaN,3.300000e+09
9,Afghanistan,1979,50.64,24.10,25.27,NaN,3.697940e+09


## Step 8: Data Validation

Quality checks before export.

In [13]:
print("=== DATA QUALITY REPORT ===")
print("\n1. Missing Data Summary:")
print(df_final.isnull().sum())
print(f"\n   Missing data %:")
print((df_final.isnull().sum() / len(df_final) * 100).round(2))

print("\n2. Sector Percentage Validation:")
df_final['Total_%'] = df_final['Primary_%'] + df_final['Secondary_%'] + df_final['Tertiary_%']
print(df_final['Total_%'].describe())

# Check for rows that don't sum to ~100%
invalid_rows = df_final[(df_final['Total_%'] < 99) | (df_final['Total_%'] > 101)]
print(f"\n   Rows with total ≠ 100% (±1%): {len(invalid_rows):,}")

print("\n3. Sample Countries - Recent Years:")
for country in ['Afghanistan', 'Syria', 'United States', 'Germany', 'China']:
    sample = df_final[(df_final['Country'] == country) & (df_final['Year'] >= 2015)]
    if len(sample) > 0:
        print(f"\n{country}:")
        print(sample[['Year', 'Primary_%', 'Secondary_%', 'Tertiary_%', 'Tourism_%']])

print("\n4. Tourism Coverage by Year:")
tourism_by_year = df_final[df_final['Tourism_%'].notna()].groupby('Year').size()
print(tourism_by_year)

# Drop the temporary Total_% column
df_final = df_final.drop(columns=['Total_%'])

=== DATA QUALITY REPORT ===

1. Missing Data Summary:
Country           0
Year              0
Primary_%       170
Secondary_%      43
Tertiary_%       49
Tourism_%      9808
GDP_USD        2773
dtype: int64

   Missing data %:
Country         0.00
Year            0.00
Primary_%       1.55
Secondary_%     0.39
Tertiary_%      0.45
Tourism_%      89.69
GDP_USD        25.36
dtype: float64

2. Sector Percentage Validation:
count    10717.000000
mean       100.024971
std          0.673156
min         79.390000
25%        100.000000
50%        100.000000
75%        100.000000
max        125.050000
Name: Total_%, dtype: float64

   Rows with total ≠ 100% (±1%): 38

3. Sample Countries - Recent Years:

Afghanistan:
    Year  Primary_%  Secondary_%  Tertiary_%  Tourism_%
45  2015      24.33         7.33       68.35        NaN
46  2016      30.53         7.59       61.88        NaN
47  2017      30.43         7.78       61.79        NaN
48  2018      28.04        10.37       61.59        NaN
49 

## Step 9: Export Master Dataset

Save to `processed-data/economics-countries-master.csv`

In [14]:
# Export to CSV
output_path = '../processed-data/economics-countries-master.csv'
df_final.to_csv(output_path, index=False)

print(f"✓ Exported to: {output_path}")
print(f"\nDataset Summary:")
print(f"  - Total records: {len(df_final):,}")
print(f"  - Countries: {df_final['Country'].nunique()}")
print(f"  - Years: {df_final['Year'].min()} - {df_final['Year'].max()} ({df_final['Year'].nunique()} years)")
print(f"  - File size: {pd.read_csv(output_path).memory_usage(deep=True).sum() / 1024:.1f} KB")

print("\n✓ Economics Countries Master dataset created successfully!")
print("\nReady to merge with conflict data using Country + Year keys.")

✓ Exported to: ../processed-data/economics-countries-master.csv

Dataset Summary:
  - Total records: 10,936
  - Countries: 220
  - Years: 1970 - 2023 (54 years)
  - File size: 1229.3 KB

✓ Economics Countries Master dataset created successfully!

Ready to merge with conflict data using Country + Year keys.
